In [26]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install datasets
!python -m spacy download en_core_web_sm

  Cloning https://github.com/alexzvk/pke.git to /tmp/pip-req-build-u3xgwjc6
  Running command git clone --filter=blob:none --quiet https://github.com/alexzvk/pke.git /tmp/pip-req-build-u3xgwjc6
  Resolved https://github.com/alexzvk/pke.git to commit 1e1459e2eb5618bf793f3ac293ed050bb5e92731
  Preparing metadata (setup.py) ... done

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [20]:
import spacy
from spacy.tokenizer import _get_regex_pattern

nlp = spacy.load("en_core_web_sm")

# Tokenization fix for in-word hyphens (e.g. 'non-linear' would be kept 
# as one token instead of default spacy behavior of 'non', '-', 'linear')
# https://spacy.io/usage/linguistic-features#native-tokenizer-additions

from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

# Modify tokenizer infix patterns
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        # ✅ Commented out regex that splits on hyphens between letters:
        # r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

In [21]:
from tqdm import tqdm
from datasets import load_dataset

benchmark = "semeval-2010-pre"
# load the inspec dataset
dataset = load_dataset('taln-ls2n/'+benchmark)

# pre-process training and test splits
train = []
for sample in tqdm(dataset['train']):
    train.append(nlp(sample["title"]+". "+sample["abstract"]))
test = []
for sample in tqdm(dataset['test']):
    test.append(nlp(sample["title"]+". "+sample["abstract"]))

100%|██████████| 100/100 [00:03<00:00, 29.72it/s]


In [22]:
from pke import compute_document_frequency, compute_lda_model
from string import punctuation


# computing LDA and DF counts

compute_document_frequency(
    documents=train,
    output_file='data/{}.df.gz'.format(benchmark),
    language='en',              # language of the input files
    normalization='stemming',   # use porter stemmer
    stoplist=list(punctuation), # stoplist (punctuation marks)
    n=5                         # compute n-grams up to 5-grams
)

compute_lda_model(
    documents=train,
    output_file="data/{}.lda.pickle.gz".format(benchmark),
    n_topics=500,               # number of topics
    language='en',              # language of the input files
    stoplist=list(punctuation), # stoplist (punctuation marks)
    normalization='stemming'    # use porter stemmer
)

In [23]:
from pke import load_document_frequency_file, load_lda_model

df = load_document_frequency_file(input_file='data/{}.df.gz'.format(benchmark))
lda_model = load_lda_model(input_file="data/{}.lda.pickle.gz".format(benchmark))

In [29]:
from pke.unsupervised import *
from timeit import default_timer as timer

outputs = {}
elapsed_times = {}
#for model in [TopicalPageRank]:
for model in [FirstPhrases, TextRank, SingleRank, TopicRank, PositionRank, MultipartiteRank, TfIdf, TopicalPageRank]:
    outputs[model.__name__] = []
    
    extractor = model()
    start = timer()
    for i, doc in enumerate(tqdm(test)):
        extractor.load_document(input=doc, language='en')
        extractor.grammar_selection(grammar="NP: {<ADJ>*<NOUN|PROPN>+}")
        if model.__name__ in ["TfIdf"]:
            extractor.candidate_weighting(df=df)
        elif model.__name__ in ["TopicalPageRank"]:
            extractor.candidate_weighting(lda_model=lda_model)
        else:
            extractor.candidate_weighting()
        outputs[model.__name__].append([u for u,v in extractor.get_n_best(n=10, stemming=True)])
    end = timer()
    elapsed_times[model.__name__] = end - start
    #print("Elapsed time for {}: {:.2f}s - {:.2f} it/s".format(model.__name__, end - start,  len(test) / (end - start)))

NameError: name 'RAKE' is not defined

In [11]:
from nltk.stem.snowball import SnowballStemmer as Stemmer
import numpy as np
    
# populates the references list with stemmed keyphrases
references = []
for sample in tqdm(dataset['test']):
    sample_keyphrases = []
    for keyphrase in sample["keyphrases"]:
        # tokenize keyphrase
        tokens = [token.text for token in nlp(keyphrase)]
        # normalize tokens using Porter's stemming
        stems = [Stemmer('porter').stem(tok.lower()) for tok in tokens]
        sample_keyphrases.append(" ".join(stems))
    references.append(sample_keyphrases)

def evaluate(top_N_keyphrases, references, cutoff=5):
    P = len(set(top_N_keyphrases[:cutoff]) & set(references)) / len(top_N_keyphrases[:cutoff])
    R = len(set(top_N_keyphrases[:cutoff]) & set(references)) / len(references)
    F = (2*P*R)/(P+R) if (P+R) > 0 else 0 
    return (P, R, F)

100%|██████████| 100/100 [00:09<00:00, 10.90it/s]


In [12]:
print("## Benchmarking on {}".format(benchmark))
print("| Model | it/s |  F@5 | F@10 |")
print("| :---- | ----:| ---: | ---: |")

# loop through the models
for model in outputs:
    
    f_scores = []
    # compute the P, R, F scores for the model
    for cutoff in [5, 10]:
        scores = []
        for i, output in enumerate(outputs[model]):
            scores.append(evaluate(output, references[i], cutoff))

        # compute the average scores
        P, R, F = np.mean(scores, axis=0)
        f_scores.append(F)
        
    print("| {}  | {:.1f} | {:.2f} | {:.2f} |".format(model,  len(test)/ elapsed_times[model], f_scores[0]*100, f_scores[1]*100))


        # print out the performance of the model
        #print("{} at {} P: {:.3f} R: {:.3f} F: {:.3f}".format(model, cutoff, avg_scores[0], avg_scores[1], avg_scores[2]))

## Benchmarking on semeval-2010-pre
| Model | it/s |  F@5 | F@10 |
| :---- | ----:| ---: | ---: |
| FirstPhrases  | 200.2 | 13.39 | 14.52 |
| TextRank  | 120.9 | 8.85 | 12.98 |
| SingleRank  | 129.4 | 11.27 | 16.36 |
| TopicRank  | 83.8 | 11.60 | 13.98 |
| PositionRank  | 126.1 | 12.60 | 16.89 |
| MultipartiteRank  | 61.4 | 12.83 | 15.36 |
| TfIdf  | 210.2 | 13.18 | 15.55 |
| TopicalPageRank  | 19.3 | 11.45 | 16.33 |


| Model | F@5 | F@10 |
| :---  | --: | ---: |
| FirstPhrases  | 0.24 | 0.29 |
| TextRank  | 0.27 | 0.34 |
| SingleRank  | 0.27 | 0.34 |
| TopicRank  | 0.25 | 0.28 |
| PositionRank  | 0.28 | 0.33 |
| MultipartiteRank  | 0.25 | 0.29 |
| TfIdf  | 0.28 | 0.35 |
| TopicalPageRank  | 0.28 | 0.34 |